In [1]:
# !pip install couchdb
# !pip install tweepy==v4.5.0 

# https://codeleading.com/article/24561506756/
# https://gist.github.com/marians/8e41fc817f04de7c4a70

In [2]:
# Ec2 config
"""
ssh -i "CCC.pem" ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com

sudo yum update -y
sudo yum install -y docker

sudo service docker start
sudo docker pull couchdb
sudo docker run -d --name my-couchdb -p 5984:5984 -e COUCHDB_USER='admin' -e COUCHDB_PASSWORD='123456' couchdb:latest

logout

ssh -i CCC.pem -L localhost:15984:172.17.0.2:5984 -N -f ec2-3-25-244-154.ap-southeast-2.compute.amazonaws.com

——————————————————

ssh -i "CCC.pem" ec2-3-25-244-154.ap-southeast-2.compute.amazonaws.com
——
sudo service docker start

sudo docker restart my-couchdb

logout
——
ssh -i CCC.pem -L localhost:15984:172.17.0.2:5984 -N -f ec2-user@ec2-3-25-244-154.ap-southeast-2.compute.amazonaws.com

http://localhost:15984/_utils

---------------------
Run on EC2

curl -O https://bootstrap.pypa.io/get-pip.py

python3 get-pip.py --user

scp -r -i CCC.pem CCC_A2 ec2-3-25-244-154.ap-southeast-2.compute.amazonaws.com:~

--------------
nohup python3 main.py &
"""

'\nssh -i "CCC.pem" ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com\n\nsudo yum update -y\nsudo yum install -y docker\n\nsudo service docker start\nsudo docker pull couchdb\nsudo docker run -d --name my-couchdb -p 5984:5984 -e COUCHDB_USER=\'admin\' -e COUCHDB_PASSWORD=\'123456\' couchdb:latest\n\nlogout\n\nssh -i CCC.pem -L localhost:15984:172.17.0.2:5984 -N -f ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com\n\n——————————————————\n\nssh -i "CCC.pem" ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com\n——\nsudo service docker start\n\nlogout\n——\nssh -i CCC.pem -L localhost:15984:172.17.0.2:5984 -N -f ec2-user@ec2-13-211-208-125.ap-southeast-2.compute.amazonaws.com\n\nhttp://localhost:15984/_utils\n\n---------------------\nRun on EC2\n\ncurl -O https://bootstrap.pypa.io/get-pip.py\n\npython3 get-pip.py --user\n\nscp -r -i CCC.pem CCC_A2 ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com:~\n\n'

In [6]:
# couchbd_settings
address = 'localhost:15984'
username = 'admin'
password = '123456'
tweets = 'raw_tweets'
user = 'user_list'
test = 'test'

In [7]:
import couchdb
# from couchbd_settings import *

# use authenticated connection to connect to couchdb server for writing to the database
def db_connect(dbname):

    couchserver = couchdb.Server('http://' + username + ':' + password + '@' + address)
    try:
      db = couchserver[dbname]
    except:
      db = couchserver.create(dbname)

    return db

tweet_db = db_connect(tweets)
user_db = db_connect(user)
test_db = db_connect(test)

def save_to_db(item_id, keyword, item, db):
    
        db[item_id] = {keyword: item}


In [5]:
# api_credentials

#Weimin 500k cap

BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAGvGbQEAAAAAMgkT5P3XP%2B3JQeU6LEvr7K8AyVM%3DsgSDIfoNhxIYMHgbWi03qYtKzgwOy5Kij5cr2cQ2QRYZhNdPzD'

# Weimin 2m cap
# CONSUMER_KEY = '2Qgue4p6wHH3dopq2ztD2ABos'
# CONSUMER_SECRET = 'JwSn0M1oYD8OSkbO2KU3uxAPWUCwiEr4GOrSSJDKvXjCbgXynt'
# ACCESS_TOKEN = '1510262166045458432-jGTjHomcIQzCmNO4qE73ByiicZJUD2'  
# ACCESS_TOKEN_SECRET = '3nYPGi7zbw2Z9LiuKqR1tzGsepuO1y8m1L5zISjIj0BqQ'
# BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAAoxbAEAAAAApib47uKVACiyXfOSAj8BELeJaf4%3DT4L0eLVTNooQRZVftkppGGnEoRqau3K2R73ZJYo3jB8TZFghZJ'

# Wen 2m cap
# BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAGjcbAEAAAAANc0%2BWz6ZlsVZCZLEHegK9luuZXo%3DBKAeeJtUU6tKm3YqNHGz8u4jEfvDhTsQ9LnUq5Sz1GofEBI3mv'
ACCESS_TOKEN = '1956431324-p8F2IRC56igWyDBPdyLyENv1hpWzurywGDR65Vu'
ACCESS_TOKEN_SECRET = 'H527uwZmSQAw5em5HavBKRScQ36Fw1siO5GimhU4vX0kZ'
CONSUMER_KEY = 'FDYIQR8qhQPl2fS61IH3ptRv5'
CONSUMER_SECRET = 'UycadDkLnlDZGPev27mDBtyfHd39Eyk6psLVPpzH5zPvP4eE7D'


In [7]:
import tweepy
import datetime
# from api_credentials import *

def authenticate():

  client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=True)

  return client

client = authenticate()


In [15]:
import json

query = 'drink OR food OR restaurant OR health OR cost of living OR petrol price OR housing OR government OR covid OR corona OR coronavirus -is:retweet'
place_id = '01864a8a64df9dc4'

def search_recent_tweets(client, query, place_id, db):

    # start_time = ''
    # end_time = ''

    for response in tweepy.Paginator(client.search_recent_tweets, 
                                     query = query, 
                                     tweet_fields = ['created_at','entities','geo','lang'],
                                     place_fields = ['geo'],
                                     expansions = 'geo.place_id,author_id',
                                     max_results = 100).flatten():
        try:
            tweet = response.data
            if tweet.get('geo', None) is not None:
                if tweet['geo'].get('place_id') == place_id:
                    tweet_id = tweet.get('id')
                    if tweet_id not in db:
                        print(tweet_id)
                        save_to_db(tweet_id, tweet, db)
                        
        except Exception as e:
            print(e)
            pass
            
search_recent_tweets(client, query, place_id, tweet_db)

Rate limit exceeded. Sleeping for 518 seconds.


KeyboardInterrupt: 

In [8]:
place_id = '01864a8a64df9dc4'

def search_timeline(id_str, place_id):
    
    for response in tweepy.Paginator(client.get_users_tweets, 
                                     id_str, 
                                     start_time = datetime.datetime(2018, 1, 1, 0, 0, 0).strftime("%Y-%m-%dT%H:%M:%SZ"),
                                     exclude='retweets',
                                     tweet_fields = ['created_at','geo','lang'],
                                     place_fields = ['geo'],
                                     expansions = 'geo.place_id,author_id',
                                     max_results = 100).flatten():
        
        tweet = response.data
        print(tweet)
        if tweet.get('geo', None) is not None:
            if tweet['geo'].get('place_id') == place_id:
                tweet_id = tweet.get('id')
                if tweet_id not in tweet_db:
                    save_to_db(str(tweet_id), 'timeline', tweet, tweet_db)

In [ ]:
# under construction - combining streaming & search timeline

import json

class TwitterStreamer(tweepy.Stream):

    def on_status(self, status):
        print(f'status_text: {status.text}')

    def on_data(self, data):
        tweet = json.loads(data)
        tweet_id = str(tweet.get('id'))
        
        if (tweet.get('retweeted_status', None) is None) and (tweet_id not in tweet_db):
            save_to_db(tweet_id, 'stream', tweet, tweet_db)
            print(f'new tweet: {tweet_id}')
            
        user_id = str(tweet['user'].get('id_str'))
        if user_id not in user_db:
            search_timeline(user_id, place_id)
            save_to_db(user_id, 'processed', 'yes', user_db) 
    
    def on_error(self, status):
        print(f'error: {status}')
        
    def on_exception(self, exception):
        print(f'exception: {exception}')
        return

stream = TwitterStreamer(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

while True:
    try:
        stream.filter(locations = [144.593741856, -38.433859306, 145.512528832, -37.5112737225])
    except:
        continue
        
stream.disconnect()

new tweet: 1513485360021454858
new tweet: 1513485368259387396


In [ ]:
# working version without search timeline

import json

class TwitterStreamer(tweepy.Stream):

    def on_status(self, status):
        print(f'status_text: {status.text}')

    def on_data(self, data):
        tweet = json.loads(data)
        tweet_id = tweet.get('id')
        
        if (tweet.get('retweeted_status', None) is None) and (str(tweet_id) not in tweet_db):
            save_to_db(str(tweet_id), 'stream', tweet, tweet_db)
            print(f'new tweet: {tweet_id}')
    
    def on_error(self, status):
        print(status)
        
    def on_exception(self, exception):
        print(f'exception: {exception}')
        return

stream = TwitterStreamer(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

while True:
    try:
        stream.filter(locations = [144.33363404800002, -38.50298801599996, 145.8784120140001, -37.17509899299995])

    except:
        continue

In [ ]:
# too inefficient without geo selection, may not be needed


"""
def query(query_item):
  query = 

return query

"""

query = 'cost of living OR petrol price OR housing -is:retweet'
# query = 'bounding_box:[-105.301758 39.964069 -105.178505 40.09455]'


import json

def search_recent_tweets(client, query, db):

    # start_time = ''
    # end_time = ''

    for response in tweepy.Paginator(client.search_recent_tweets, 
                                query = query, 
                                tweet_fields = ['created_at', 'geo', 'lang'],
                                user_fields = ['id', 'username'],
                                place_fields = ['geo'],
                                expansions = 'geo.place_id,author_id').flatten(1): #delete number if not testing

        try:
            tweet = json.loads(json.dumps(response.data))
            tweet_id = str(tweet.get('id'))
            if tweet_id not in db:
                save_to_db(tweet_id, tweet, db)
        except Exception as e:
            print(e)
            pass
            
search_recent_tweets(client, query, tweet_db)

In [ ]:
# import requests

# def bearer_oauth(r):
#     """
#     Method required by bearer token authentication.
#     """

#     r.headers["Authorization"] = f"Bearer {BEARER_TOKEN}"
#     # r.headers["User-Agent"] = "v2FilteredStreamPython"
#     return r

# url = 'https://api.twitter.com/2/users/215562059/tweets?expansions=geo.place_id&tweet.fields=id,created_at,text,author_id,geo,lang&place.fields=id,name,country_code,place_type,full_name,country,contained_within,geo'

# response = requests.get(
#     url,
#     auth=bearer_oauth
# )

# tweet = response

# print(tweet)

# tweet = response.json()
# print(tweet['includes'])

In [ ]:
# #start_time =
# #end_time = 

# max_results = 5


# response = client.get_users_tweets(id = '215562059',
#                                         exclude='retweets',
#                                         tweet_fields = ['created_at','geo','lang'],
#                                         place_fields = ['geo'],
#                                         expansions = 'geo.place_id,author_id',
#                                         max_results = max_results)



# # combine response.data & response.includes

# while 'next_token' in response.meta:
#     response = client.get_users_tweets(id = '215562059',
#                                         exclude='retweets',
#                                         tweet_fields = ['created_at','geo','lang'],
#                                         place_fields = ['geo'],
#                                         expansions = 'geo.place_id,author_id',
#                                         pagination_token = response.meta['next_token'],
#                                         max_results = 100)
    
#     print(f'get page: {response.meta["next_token"]}')


    

In [ ]:
# import tweepy
# # from twitter_credentials import *
# from tweepy.streaming import StreamListener
# from tweepy import OAuthHandler
# from tweepy import Stream

# def authenticate():

#   auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
#   auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

#   return tweepy.API(auth, wait_on_rate_limit = True)

# api = authenticate()